## CSV解析器

当您想要返回以逗号分隔的项目列表时，可以使用此输出解析器。



In [2]:
from langchain_openai import ChatOpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)

chat.invoke("请问2只兔子有多少条腿？")

AIMessage(content='每只兔子有4条腿，所以2只兔子共有8条腿。')

In [33]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

# format_instructions = output_parser.get_format_instructions()
format_instructions = "您的响应应该是csv格式的逗号分隔值的列表，例如：`内容1, 内容2, 内容3`"
prompt = PromptTemplate(
    template="{format_instructions}\n请列出五个 {subject}.",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)



chain = prompt | chat | output_parser

In [34]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [30]:
chain.invoke({"subject": "冰淇淋口味"})

['香草', '巧克力', '草莓', '抹茶', '薄荷']

In [28]:
prompt.invoke({"subject": "冰淇淋口味"})

StringPromptValue(text='您的响应应该是csv格式的逗号分隔值的列表，例如：`内容1, 内容2, 内容3`\n请列出五个 冰淇淋口味.')

## 日期时间解析器
此 OutputParser 可用于将 LLM 输出解析为日期时间格式。

In [35]:
from langchain.output_parsers import DatetimeOutputParser
output_parser = DatetimeOutputParser()
template = """回答用户的问题:

{question}

{format_instructions}"""


format_instructions=output_parser.get_format_instructions()

format_instructions='''响应的格式用日期时间字符串:“%Y-%m-%dT%H:%M:%S.%fZ”。

示例: 1898-05-31T06:59:40.248940Z, 1808- 10-20T01:56:09.167633Z、0226-10-17T06:18:24.192024Z

仅返回此字符串，没有其他单词！'''
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions":format_instructions },
)

In [36]:
format_instructions

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1898-05-31T06:59:40.248940Z, 1808-10-20T01:56:09.167633Z, 0226-10-17T06:18:24.192024Z\n\nReturn ONLY this string, no other words!"

In [37]:
chain = prompt | chat | output_parser

In [39]:
output = chain.invoke({"question": "比特币是什么时候创立的？"})
output

datetime.datetime(2009, 1, 3, 18, 15)

## Enum parser 枚举解析器

In [41]:
from langchain.output_parsers.enum import EnumOutputParser

In [72]:
from enum import Enum

class Colors(Enum):
    RED = "红色"
    BROWN = "棕色"
    BLACK = "黑色"
    WHITE = "白色"
    YELLOW = "黄色"
    

In [73]:
parser = EnumOutputParser(enum=Colors)

In [74]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

promptTemplate = PromptTemplate.from_template(
    """{person}的皮肤主要是什么颜色？

{instructions}"""
)

# instructions=parser.get_format_instructions()
instructions = "响应的结果请选择以下选项之一：红色、棕色、黑色、白色、黄色。不要有其他的内容"
prompt = promptTemplate.partial(instructions=instructions)
chain = prompt | chat | parser

In [75]:
instructions=parser.get_format_instructions()
instructions

'Select one of the following options: 红色, 棕色, 黑色, 白色, 黄色'

In [76]:
chain.invoke({"person": "亚洲人"})

<Colors.YELLOW: '黄色'>